In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'São Paulo - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH,São Paulo - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,437.972720,0.314974,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,0.812723,782.111
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,434.363720,0.317328,1.036092e+09,4.732994e+07,23.901472,9.822666e+08,8.304194,0.812838,789.891
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,433.350868,0.319840,1.037280e+09,4.735047e+07,23.907385,9.826203e+08,8.298053,0.812953,774.309
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,431.825332,0.322104,1.038469e+09,4.737101e+07,23.913298,9.829741e+08,8.291912,0.813068,782.071
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,433.287298,0.324320,1.039658e+09,4.739155e+07,23.919210,9.833278e+08,8.285771,0.813183,841.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.306495,0.590649,NaN,NaN,NaN,NaN,NaN,NaN,1112.914
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.812591,0.588569,NaN,NaN,NaN,NaN,NaN,NaN,1028.144
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.856452,0.585196,NaN,NaN,NaN,NaN,NaN,NaN,996.449
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.215930,0.581094,NaN,NaN,NaN,NaN,NaN,NaN,1024.272


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.209684,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,0.784878
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.192369,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,0.748669
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.178463,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,0.712461
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.166182,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,0.676253


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: São Paulo - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.608419,1.023937,1.193677,-1.393041,-1.206846,0.733610,1.413038,1.566441
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.653197,1.033712,1.195577,-1.411276,-1.217300,0.727991,1.392752,1.539489
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.699576,1.049101,1.197477,-1.429511,-1.227755,0.722372,1.372465,1.512536
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.757794,1.059564,1.199378,-1.447746,-1.238210,0.716753,1.352178,1.485583


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
157    757.915
158    923.634
159    756.663
160    872.715
161    844.988
Name: São Paulo - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,1.357021,0.718372,0.835331,-0.070173,-0.126212,1.060884,0.965416,0.617332
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,1.320237,0.803494,0.853864,-0.120590,-0.162580,1.058281,0.995407,0.624883
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.259517,0.892135,0.872396,-0.171007,-0.198948,1.055678,1.025397,0.632435
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.196227,0.945408,0.890929,-0.221425,-0.235316,1.053074,1.055388,0.639987
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.135312,0.989786,0.909462,-0.271842,-0.271684,1.050471,1.085379,0.647538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.209684,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,0.784878
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.192369,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,0.748669
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.178463,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,0.712461
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.166182,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,0.676253


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(36, activation='tanh', 
                                 return_sequences=True), 
                                 input_shape=(train_input.shape[1], 
                                              train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(180, activation='tanh')),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[4060482897, 3981323922, 1430960087, 400367265, 3753840055, 2330519013, 305312116, 2530165622, 1876428836, 1337411009, 3726092382, 1865708458, 1126941353, 453090551, 4085810057, 1555581399, 1343613941, 1271798674, 2512057041, 444940501, 3900109946, 3546644411, 1460598311, 2215090735, 3164172307, 595750879, 3766895664, 1018738548, 298852520, 3820652490, 1120482959, 2764376601, 1432135442, 3598122063, 1587461829, 2519678635, 3776036718, 3819822826, 2612429213, 577576769, 3343386922, 4168387885, 2313097643, 1095362633, 3959704845, 23905477, 2799129440, 3891080121, 1956434447, 901393281]


Step: 0 ___________________________________________
val_loss: 8064.8837890625
winner_seed: 4060482897


Step: 1 ___________________________________________
val_loss: 8674.0234375


Step: 2 ___________________________________________
val_loss: 6499.46484375
winner_seed: 1430960087


Step: 3 ___________________________________________
val_loss: 9346.3701171875


Step: 4 ____________________________________

In [35]:
trained_model, history = bidirectional_lstm_model(reshaped_train, 
                                                  reshaped_target, 
                                                  want_verbose=1, 
                                                  seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 3s 241ms/step - loss: 796926.1875 - val_loss: 19224.7051
Epoch 2/10000
4/4 [==============================] - 0s 17ms/step - loss: 35543.3555 - val_loss: 27793.6992
Epoch 3/10000
4/4 [==============================] - 0s 17ms/step - loss: 55873.3828 - val_loss: 34185.7930
Epoch 4/10000
4/4 [==============================] - 0s 16ms/step - loss: 61189.4180 - val_loss: 108803.1172
Epoch 5/10000
4/4 [==============================] - 0s 17ms/step - loss: 125106.3750 - val_loss: 152556.7812
Epoch 6/10000
4/4 [==============================] - 0s 14ms/step - loss: 176201.5312 - val_loss: 38197.1055
Epoch 7/10000
4/4 [==============================] - 0s 14ms/step - loss: 86288.3828 - val_loss: 32714.4375
Epoch 8/10000
4/4 [==============================] - 0s 14ms/step - loss: 84780.1172 - val_loss: 142330.1719
Epoch 9/10000
4/4 [==============================] - 0s 14ms/step - loss: 210515.3906 - val_loss: 65385.8828
Epoch 10/10000
4/4 [

4/4 [==============================] - 0s 15ms/step - loss: 11590.0420 - val_loss: 16443.4648
Epoch 77/10000
4/4 [==============================] - 0s 14ms/step - loss: 11338.6484 - val_loss: 12288.7129
Epoch 78/10000
4/4 [==============================] - 0s 15ms/step - loss: 12407.1123 - val_loss: 16020.3096
Epoch 79/10000
4/4 [==============================] - 0s 13ms/step - loss: 11259.4443 - val_loss: 13240.1953
Epoch 80/10000
4/4 [==============================] - 0s 13ms/step - loss: 13376.4824 - val_loss: 8149.1646
Epoch 81/10000
4/4 [==============================] - 0s 13ms/step - loss: 12411.2393 - val_loss: 8807.0488
Epoch 82/10000
4/4 [==============================] - 0s 13ms/step - loss: 11069.0166 - val_loss: 9579.9170
Epoch 83/10000
4/4 [==============================] - 0s 15ms/step - loss: 10968.1475 - val_loss: 8999.4111
Epoch 84/10000
4/4 [==============================] - 0s 16ms/step - loss: 11663.4629 - val_loss: 17404.2461
Epoch 85/10000
4/4 [==================

Epoch 151/10000
4/4 [==============================] - 0s 21ms/step - loss: 13014.4551 - val_loss: 14447.7959
Epoch 152/10000
4/4 [==============================] - 0s 18ms/step - loss: 12844.0850 - val_loss: 15725.8643
Epoch 153/10000
4/4 [==============================] - 0s 19ms/step - loss: 13630.1855 - val_loss: 17537.0176
Epoch 154/10000
4/4 [==============================] - 0s 18ms/step - loss: 12931.3154 - val_loss: 18521.5117
Epoch 155/10000
4/4 [==============================] - 0s 19ms/step - loss: 13865.1543 - val_loss: 17289.4590
Epoch 156/10000
4/4 [==============================] - 0s 19ms/step - loss: 12491.8359 - val_loss: 14924.6934
Epoch 157/10000
4/4 [==============================] - 0s 16ms/step - loss: 13727.8457 - val_loss: 32048.3184
Epoch 158/10000
4/4 [==============================] - 0s 16ms/step - loss: 16746.5332 - val_loss: 25484.5332
Epoch 159/10000
4/4 [==============================] - 0s 14ms/step - loss: 16011.1748 - val_loss: 26225.3574
Epoch 160/

4/4 [==============================] - 0s 17ms/step - loss: 10436.7676 - val_loss: 11972.0762
Epoch 226/10000
4/4 [==============================] - 0s 18ms/step - loss: 12450.8252 - val_loss: 12159.8115
Epoch 227/10000
4/4 [==============================] - 0s 19ms/step - loss: 11169.1943 - val_loss: 19042.6289
Epoch 228/10000
4/4 [==============================] - 0s 18ms/step - loss: 12938.8252 - val_loss: 12356.9795
Epoch 229/10000
4/4 [==============================] - 0s 16ms/step - loss: 10478.1055 - val_loss: 11067.2646
Epoch 230/10000
4/4 [==============================] - 0s 16ms/step - loss: 11023.1084 - val_loss: 11104.7891
Epoch 231/10000
4/4 [==============================] - 0s 17ms/step - loss: 13149.7734 - val_loss: 17177.2109
Epoch 232/10000
4/4 [==============================] - 0s 18ms/step - loss: 12127.7598 - val_loss: 12728.7266
Epoch 233/10000
4/4 [==============================] - 0s 16ms/step - loss: 11111.7061 - val_loss: 29250.3848
Epoch 234/10000
4/4 [=====

4/4 [==============================] - 0s 19ms/step - loss: 11404.2764 - val_loss: 25424.7207
Epoch 300/10000
4/4 [==============================] - 0s 19ms/step - loss: 11758.9453 - val_loss: 11155.5693
Epoch 301/10000
4/4 [==============================] - 0s 19ms/step - loss: 10725.2998 - val_loss: 12795.7051
Epoch 302/10000
4/4 [==============================] - 0s 17ms/step - loss: 10287.7334 - val_loss: 12717.8555
Epoch 303/10000
4/4 [==============================] - 0s 20ms/step - loss: 12128.4023 - val_loss: 13211.2139
Epoch 304/10000
4/4 [==============================] - 0s 20ms/step - loss: 8243.9941 - val_loss: 10207.9229
Epoch 305/10000
4/4 [==============================] - 0s 20ms/step - loss: 12753.4229 - val_loss: 11209.6963
Epoch 306/10000
4/4 [==============================] - 0s 19ms/step - loss: 9771.0410 - val_loss: 20311.4102
Epoch 307/10000
4/4 [==============================] - 0s 21ms/step - loss: 9270.0859 - val_loss: 15177.0342
Epoch 308/10000
4/4 [========

...

Epoch 674/10000
4/4 [==============================] - 0s 18ms/step - loss: 8818.6621 - val_loss: 20379.9160
Epoch 675/10000
4/4 [==============================] - 0s 16ms/step - loss: 11209.0195 - val_loss: 9620.8320
Epoch 676/10000
4/4 [==============================] - 0s 17ms/step - loss: 10957.4238 - val_loss: 9204.3408
Epoch 677/10000
4/4 [==============================] - 0s 16ms/step - loss: 8610.2627 - val_loss: 25606.7891
Epoch 678/10000
4/4 [==============================] - 0s 19ms/step - loss: 11522.1641 - val_loss: 8189.3726
Epoch 679/10000
4/4 [==============================] - 0s 20ms/step - loss: 11349.1748 - val_loss: 18413.7188
Epoch 680/10000
4/4 [==============================] - 0s 18ms/step - loss: 10398.1152 - val_loss: 21058.2207
Epoch 681/10000
4/4 [==============================] - 0s 16ms/step - loss: 9491.3896 - val_loss: 20353.3867
Epoch 682/10000
4/4 [==============================] - 0s 17ms/step - loss: 11677.9385 - val_loss: 17740.2773
Epoch 683/10000


Epoch 749/10000
4/4 [==============================] - 0s 18ms/step - loss: 13025.3428 - val_loss: 12653.7705
Epoch 750/10000
4/4 [==============================] - 0s 19ms/step - loss: 11552.2031 - val_loss: 9082.6846
Epoch 751/10000
4/4 [==============================] - 0s 17ms/step - loss: 10434.2949 - val_loss: 19189.9355
Epoch 752/10000
4/4 [==============================] - 0s 18ms/step - loss: 12070.1318 - val_loss: 15846.9229
Epoch 753/10000
4/4 [==============================] - 0s 18ms/step - loss: 11550.4199 - val_loss: 12322.9746
Epoch 754/10000
4/4 [==============================] - 0s 18ms/step - loss: 12943.1455 - val_loss: 22455.0977
Epoch 755/10000
4/4 [==============================] - 0s 19ms/step - loss: 9334.6680 - val_loss: 13018.1045
Epoch 756/10000
4/4 [==============================] - 0s 17ms/step - loss: 9425.2832 - val_loss: 13675.4629
Epoch 757/10000
4/4 [==============================] - 0s 20ms/step - loss: 10089.8633 - val_loss: 18882.0156
Epoch 758/100

Epoch 824/10000
4/4 [==============================] - 0s 17ms/step - loss: 7646.4238 - val_loss: 18371.0312
Epoch 825/10000
4/4 [==============================] - 0s 15ms/step - loss: 9687.7012 - val_loss: 8785.1885
Epoch 826/10000
4/4 [==============================] - 0s 17ms/step - loss: 10077.9893 - val_loss: 22002.4414
Epoch 827/10000
4/4 [==============================] - 0s 19ms/step - loss: 11502.3516 - val_loss: 11731.7236
Epoch 828/10000
4/4 [==============================] - 0s 18ms/step - loss: 10524.8242 - val_loss: 19093.1914
Epoch 829/10000
4/4 [==============================] - 0s 16ms/step - loss: 12033.8105 - val_loss: 22876.4219
Epoch 830/10000
4/4 [==============================] - 0s 19ms/step - loss: 10333.8525 - val_loss: 15973.5146
Epoch 831/10000
4/4 [==============================] - 0s 17ms/step - loss: 10153.7266 - val_loss: 14064.1855
Epoch 832/10000
4/4 [==============================] - 0s 18ms/step - loss: 10004.5254 - val_loss: 18698.8633
Epoch 833/100

Epoch 899/10000
4/4 [==============================] - 0s 18ms/step - loss: 8674.4141 - val_loss: 14977.4121
Epoch 900/10000
4/4 [==============================] - 0s 18ms/step - loss: 10049.7500 - val_loss: 8982.1113
Epoch 901/10000
4/4 [==============================] - 0s 17ms/step - loss: 13117.9082 - val_loss: 12886.7783
Epoch 902/10000
4/4 [==============================] - 0s 18ms/step - loss: 10212.0146 - val_loss: 40708.3320
Epoch 903/10000
4/4 [==============================] - 0s 18ms/step - loss: 13052.5322 - val_loss: 28812.1543
Epoch 904/10000
4/4 [==============================] - 0s 18ms/step - loss: 9922.7500 - val_loss: 9887.6699
Epoch 905/10000
4/4 [==============================] - 0s 17ms/step - loss: 11217.3916 - val_loss: 10136.3750
Epoch 906/10000
4/4 [==============================] - 0s 16ms/step - loss: 9343.9092 - val_loss: 15462.9580
Epoch 907/10000
4/4 [==============================] - 0s 16ms/step - loss: 8625.2979 - val_loss: 10587.7549
Epoch 908/10000


Epoch 974/10000
4/4 [==============================] - 0s 14ms/step - loss: 12078.6367 - val_loss: 11998.5117
Epoch 975/10000
4/4 [==============================] - 0s 14ms/step - loss: 10522.2412 - val_loss: 12499.0479
Epoch 976/10000
4/4 [==============================] - 0s 14ms/step - loss: 12895.8525 - val_loss: 10822.2676
Epoch 977/10000
4/4 [==============================] - 0s 14ms/step - loss: 12662.6758 - val_loss: 13974.1758
Epoch 978/10000
4/4 [==============================] - 0s 14ms/step - loss: 11521.0879 - val_loss: 11822.2070
Epoch 979/10000
4/4 [==============================] - 0s 14ms/step - loss: 13622.2646 - val_loss: 8743.8330
Epoch 980/10000
4/4 [==============================] - 0s 12ms/step - loss: 15448.9346 - val_loss: 11632.5459
Epoch 981/10000
4/4 [==============================] - 0s 12ms/step - loss: 13429.2510 - val_loss: 29090.6582
Epoch 982/10000
4/4 [==============================] - 0s 13ms/step - loss: 12006.8203 - val_loss: 16695.4727
Epoch 983/1

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 21ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,800.775574,846.575562,830.81311,851.42395,853.591064,811.73877,789.891357,810.354736,802.013672,858.295898,824.785889,792.800415,909.130798,829.353394,885.116821,889.521301,849.063721,900.596497,872.651184,810.544189,880.992554,831.715332,776.075439,892.977051,881.832031,801.527832,866.458984,871.312195,850.875366,858.157898
Target,910.653,937.696,871.611,795.853,804.767,712.733,763.665,727.397,855.245,855.042,648.867,1003.807,855.104,913.329,849.262,897.883,860.059,771.417,860.232,773.607,790.146,886.929,941.6,817.301,957.289,978.99,906.956,980.948,872.046,692.232
Error,109.877441,91.120422,40.797913,55.570923,48.824036,99.005798,26.226379,82.957764,53.231323,3.253906,175.918884,211.006592,54.026794,83.975586,35.854797,8.361694,10.9953,129.179504,12.419189,36.937195,90.846558,55.213684,165.524536,75.676025,75.45697,177.462158,40.497009,109.635803,21.170654,165.925903


In [38]:
display(mae)
display(mape)

78.23169

0.09349942

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model,
                                             reshaped_test, 
                                             reshaped_test_target,
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[9873.06]] - Target[9887.336000000001]| =  Error: [[14.276367]]; MAPE:[[0.0014439]]
1/1 [==============================] - 0s 17ms/step
Ano-0: |Prediction[[10327.738]] - Target[10216.868999999999]| =  Error: [[110.86914]]; MAPE:[[0.01085158]]
1/1 [==============================] - 0s 16ms/step
Ano-5: |Prediction[[5130.164]] - Target[5388.461]| =  Error: [[258.29688]]; MAPE:[[0.04793518]]


[array([[14.276367]], dtype=float32),
 array([[110.86914]], dtype=float32),
 array([[258.29688]], dtype=float32)]

127.814125

0.020076888